In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing import pipeline2

### 데이터 전처리

In [2]:
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
# df = pd.read_csv(file_path, encoding ='cp949')
df = pd.read_csv(file_path)

C:\Users\user\AppData\Local\Temp\ipykernel_696\2437011318.py:3: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [3]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
# df = pd.read_csv(file_path, encoding ='cp949')
df = pd.read_csv(file_path)

# ✅ 모집공고일을 datetime 형식으로 변환
df["모집공고일"] = pd.to_datetime(df["모집공고일"])

# ✅ 2025-03-12 이상인 매물만 필터링
df = df[df["모집공고일"] < "2025-01-01"]

# 데이터 전처리````
preprocessing_pipeline = pipeline(type='train')
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "250318-03-삭제예정"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_696\2756494526.py:4: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\developments\hk-project\src\data_preprocessing.py:146: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(csv_path, encoding="cp949")
c:\Users\user\developments\hk-project\src\data_preprocessing.py:329: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original o

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2967 entries, 0 to 17106
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        2967 non-null   int64  
 1   공급규모          2967 non-null   int64  
 2   투기과열지구        2967 non-null   object 
 3   조정대상지역        2967 non-null   object 
 4   분양가상한제        2967 non-null   object 
 5   정비사업          2967 non-null   object 
 6   공공주택지구        2967 non-null   object 
 7   대규모택지개발지구     2967 non-null   object 
 8   수도권내민영공공주택지구  2967 non-null   object 
 9   공급세대수         2967 non-null   int64  
 10  순위            2967 non-null   object 
 11  거주지역          2967 non-null   object 
 12  접수건수          2967 non-null   int64  
 13  경쟁률           2967 non-null   float64
 14  최저당첨가점        2967 non-null   float64
 15  토픽 1          2967 non-null   float64
 16  토픽 2          2967 non-null   float64
 17  토픽 3          2967 non-null   float64
 18  토픽 4          2967 non-null   fl

### 모델 학습 및 평가

In [5]:
# 전처리된 데이터 가져오기

# file_version = "250317-01"
# file_path = f"./storage/train_data/t``ain-{file_version}.csv"
# df = pd.read_csv(file_path, encoding='cp949')

df = pd.read_csv("./storage/train_data/train-250318-03-삭제예정.csv", encoding = 'cp949')


In [6]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42,
    stratify=df["투기과열지구"]  # 분포 맞추기 위해 미달여부로 stratify
)

# 분양가상한제, 투기과열지구, 조정대상지역 -> 고려해보기염

In [7]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '공공주택지구',
       '대규모택지개발지구', '수도권내민영공공주택지구', '공급세대수', '순위', '거주지역', '접수건수', '경쟁률',
       '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5', '토픽 6', '토픽 7', '전용면적',
       '공급금액(최고가 기준)', '법정동코드', '거래금액(만원)', '전용면적당 시세차익'],
      dtype='object')
Index(['공급규모', '공급세대수', '접수건수', '경쟁률', '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5',
       '토픽 6', '토픽 7', '전용면적', '공급금액(최고가 기준)', '법정동코드', '거래금액(만원)',
       '전용면적당 시세차익', '투기과열지구_N', '투기과열지구_Y', '조정대상지역_N', '조정대상지역_Y',
       '분양가상한제_N', '분양가상한제_Y', '정비사업_N', '정비사업_Y', '공공주택지구_N', '공공주택지구_Y',
       '대규모택지개발지구_N', '대규모택지개발지구_Y', '수도권내민영공공주택지구_N', '수도권내민영공공주택지구_Y',
       '순위_1순위', '거주지역_기타지역', '거주지역_해당지역', '공급지역코드_100', '공급지역코드_400',
       '공급지역코드_410'],
      dtype='object')
⚠️ Warning: 새로운 법정동코드 발견 {'4146510300.0', '1123010700.0', '4139012700.0', '4148025327.0', '4111514000.0', '4161011100.0', '4137012200.0'}. 'unknown'으로 대체합니다.
Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '

c:\Users\user\anaconda3\envs\venv\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\user\anaconda3\envs\venv\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [9]:
joblib.dump(feature_pipeline, "./storage/trained_pipeline/pipeline_0.0.1.pkl")

['./storage/trained_pipeline/pipeline_0.0.1.pkl']

In [10]:
X_train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2373 entries, 2316 to 1827
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   공급규모            2373 non-null   float64
 1   공급세대수           2373 non-null   float64
 2   접수건수            2373 non-null   float64
 3   경쟁률             2373 non-null   float64
 4   토픽 1            2373 non-null   float64
 5   토픽 2            2373 non-null   float64
 6   토픽 3            2373 non-null   float64
 7   토픽 4            2373 non-null   float64
 8   토픽 5            2373 non-null   float64
 9   토픽 6            2373 non-null   float64
 10  토픽 7            2373 non-null   float64
 11  전용면적            2373 non-null   float64
 12  공급금액(최고가 기준)    2373 non-null   float64
 13  법정동코드           2373 non-null   int64  
 14  거래금액(만원)        2064 non-null   float64
 15  전용면적당 시세차익      2373 non-null   float64
 16  투기과열지구_N        2373 non-null   bool   
 17  투기과열지구_Y        2373 non-null   boo

In [11]:
# TODO: 여러 모델 학습 및 하이퍼파라미터 최적화

model = RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    random_state=42
)

# 모델 학습
model.fit(X_train_transformed, y_train)

# 예측
y_pred = model.predict(X_test_transformed)

# 평가 (RMSE와 R^2 계산)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse) # 낮을수록 좋음
print("R^2 Score:", r2) # 1에 가까울수록 좋음

RMSE: 5.382319259023994
R^2 Score: 0.8738427084250787


### 모델 저장 

In [12]:
# 모델 저장 및 로드

version = '0.0.2'

# 모델 저장
joblib.dump(model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_0.0.2.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)